In [ ]:
pip install pycaret

In [ ]:
# for basic mathematics operation 
import numpy as np
import pandas as pd
from pandas import plotting
import datetime

# for visualizations
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
import datetime as dt
import missingno as msno

# for interactive visualizations
import plotly.offline as py
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
from plotly import tools
init_notebook_mode(connected = True)
import plotly.figure_factory as ff
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

#Word Cloud
from PIL import Image
import requests
from io import BytesIO
from wordcloud import WordCloud
from wordcloud import ImageColorGenerator
from textblob import TextBlob
from pycaret.regression import *

# MissingNo, Plotly, Cufflinks, wordcloud, textblob, pycaret

import os
print(os.listdir('../input/'))

In [ ]:
# Dataset Load
data = pd.read_csv('../input/summer-products-and-sales-in-ecommerce-wish/summer-products-with-rating-and-performance_2020-08.csv')
# To display first five rows
data.head()

In [ ]:
#drop merchant_title, merchant_info_subtitle
data.drop(['merchant_title', 'merchant_info_subtitle','title'], axis = 1, inplace = True)
print("Columns drop successfully...")

In [ ]:
#MOST SOLD PRODUCTS - Top 3
most_sold_product = data['units_sold'].idxmax()
print("Prodcut with highest sales irrespective of price: \n")
print(data.iloc[most_sold_product].head())
print('\n')
response1 = requests.get(data[data['units_sold'] == 100000].product_picture.tolist()[0]) 

#printing the thumbnail of that prodcuts
Image.open(BytesIO(response1.content))

In [ ]:
#most_sold_product = data['units_sold'].idxmax()
#print("Prodcut with highest sales irrespective of price: \n")
print(data.iloc[18].head())
print('\n')
response2 = requests.get(data[data['units_sold'] == 100000].product_picture.tolist()[1]) 
Image.open(BytesIO(response2.content))

In [ ]:
print(data.iloc[19].head())
print('\n')
#print("Prodcut with highest sales irrespective of price: \n")
#print(data.iloc[most_sold_product].head())
#print('\n')
response3 = requests.get(data[data['units_sold'] == 100000].product_picture.tolist()[2])
Image.open(BytesIO(response3.content))

In [ ]:
# Gender Collection (Male and Female)
Total_male = round(data.title_orig.str.count("Men").sum()/len(data)*100, 3)
Total_female = round(data.title_orig.str.count("Women").sum()/len(data)*100, 3) 

fig = go.Figure()
fig.add_trace(go.Indicator(mode = "number+delta",
                             value = Total_male,
                             title = {"text": "Men Collection in %",
                                      "font" : {'color': 'rgb(58, 171, 163)', 'size': 25, 'family': 'Raleway'}},
                             number = {'font': {'color': 'rgb(58, 171, 163)', 'size': 25, 'family': 'Raleway'}},
                             domain = {'row': 0, 'column': 0}))

fig.add_trace(go.Indicator(mode = "number+delta",
                             value = Total_female,
                             title = {"text": "Women Collection in %",
                                      "font" : {'color': 'rgb(0, 0, 0)', 'size': 35, 'family': 'Raleway'}},
                             number = {'font': {'color': 'rgb(0, 0, 0)', 'size': 35, 'family': 'Raleway'}},
                             domain = {'row': 0, 'column': 1}))
fig.update_layout(grid = {'rows': 1, 'columns': 2, 'pattern': 'independent'})
fig.show()

In [ ]:
#RETAIL PRICE #FINAL PRICE- CHARGED TO CUSTOMER
plt.subplot(1, 2, 1)
(data['retail_price']).plot.hist(bins=50, figsize=(12, 6), edgecolor = 'white', range = [0, 250])
plt.xlabel('retail_price', fontsize=12)
plt.title('Retail Price Distribution', fontsize=12)
plt.subplot(1, 2, 2)
np.log(data['retail_price']+1).plot.hist(bins=50, figsize=(12,6), edgecolor='white')
plt.xlabel('log(retail_price+1)', fontsize=12)
plt.title('Retail Price Distribution', fontsize=12)

In [ ]:
#IS THERE ANY RELATION BETWEEN REATIL PRICE AND AD_BOOST
#uses_ad_boosts

ad_boost_by_buyer_no = data.loc[data['uses_ad_boosts'] == 0, 'retail_price']
ad_boost_by_seller_yes = data.loc[data['uses_ad_boosts'] == 1, 'retail_price']
fig, ax = plt.subplots(figsize=(18,8))
ax.hist(np.log(ad_boost_by_seller_yes+1), color='#0b2adb', alpha=1.0, bins=50,
       label='if there is ad-boost')
ax.hist(np.log(ad_boost_by_buyer_no+1), color='#d4db0b', alpha=0.7, bins=50,
       label='if there is no ad-boost')
plt.xlabel('log(Retail price + 1)', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.title('Retail Price Distribution by Ad-Boost', fontsize=15)
plt.tick_params(labelsize=12)
plt.legend()
plt.show()
print('The average price is EUR {}'.format(round(ad_boost_by_seller_yes.mean(), 2)), 'if there is ad-boost');
print('The average price is EUR {}'.format(round(ad_boost_by_buyer_no.mean(), 2)), 'if there is no ad-boost')

In [ ]:
#badge_product_quality
#Relationship between Product Quality and Price

product_quality_no = data.loc[data['badge_product_quality'] == 0, 'retail_price']
product_quality_yes = data.loc[data['badge_product_quality'] == 1, 'retail_price']
fig, ax = plt.subplots(figsize=(18,8))
ax.hist(product_quality_yes, color='#111212', alpha=1.0, bins=50, range = [0, 100],
       label='Prodcut Quality Badge Present')
ax.hist(product_quality_no, color='#02f0c8', alpha=0.7, bins=50, range = [0, 100],
       label='Prodcut Quality Badge Absent')
plt.xlabel('Retail Price', fontsize=12)
plt.ylabel('frequency', fontsize=12)
plt.title('Price Distribution by Product Quality Badge', fontsize=15)
plt.tick_params(labelsize=12)
plt.legend()
plt.show()

In [ ]:
#Successful Products(we will consider the low limit where we can say that product on that website is successful or not. Putting that limit to 1000.)
data['success_product'] = np.nan
for i in data.index:
    if data['units_sold'].loc[i] >= 1000:
        data['success_product'].loc[i] = 1
    else:
        data['success_product'].loc[i] = 0
        
#FIND AVG RATING FOR SUCCESSFUL PRODUCT AND UNSUCCESSFUL PRODUCT
success_rating = float("{:.3f}".format(data[data["success_product"] == 1.0].rating.mean()))
unsuccess_rating = float("{:.3f}".format(data[data["success_product"] == 0.0].rating.mean()))

print("The average rating of the successful product is: "+ str(success_rating) + '\n')
print("The average rating of the unsuccessful product is: "+ str(unsuccess_rating))



In [ ]:
#WORD CLOUD OF TAGS
tags_total = data['tags'].dropna().tolist()
tags_final=(" ").join(tags_total)
response = requests.get('https://www.incrediblelab.com/wp-content/uploads/2017/04/wish-logo.jpg')
char_mask = np.array(Image.open(BytesIO(response.content)))
image_colors = ImageColorGenerator(char_mask)
plt.figure(figsize = (13,7))

wc = WordCloud(background_color="Black", max_words=200,
               width=400, height=400, mask=char_mask, random_state=1).generate(tags_final)
# to recolour the image
plt.imshow(wc.recolor(color_func=image_colors))

REGRESSION

In [ ]:
#take few columns and made new df
df=data[['title_orig','tags', 'retail_price','units_sold', 'price', 'rating', 'rating_count',
         'rating_five_count', 'rating_one_count', 'rating_three_count', 'rating_two_count', 'rating_four_count',
         'badge_local_product', 'product_variation_inventory', 'shipping_option_price', 
         'merchant_rating_count', 'merchant_rating']]
df.head(3)

In [ ]:
# units_sold -> Y Variable
#pip install pycaret
from pycaret.regression import *

In [ ]:
df['units_sold'] = df['units_sold'].astype('float')

In [ ]:
df = df.dropna()
X = df.drop(['units_sold', 'title_orig', 'tags'], axis = 1)
y = df['units_sold']

In [ ]:
X.info()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesRegressor

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
reg = ExtraTreesRegressor(n_estimators=50, random_state=42).fit(X_train, y_train)

In [ ]:
reg

In [ ]:
reg.score(X_test, y_test)

In [ ]:
reg.get_params(deep=True)

In [ ]:
# Computing the importance of each feature
feature_importance = reg.feature_importances_
  
# Normalizing the individual importances
feature_importance_normalized = np.std([tree.feature_importances_ for tree in 
                                        reg.estimators_],
                                        axis = 0)

plt.figure(figsize = (33,12))
plt.bar(X.columns, feature_importance_normalized)
plt.xlabel('Feature Labels')
plt.ylabel('Feature Importances')
plt.title('Comparison of different Feature Importances')
plt.show()

In [ ]:
feature_importance_normalized

In [ ]:
exp_reg = setup(df, target = 'units_sold', silent = True)

In [ ]:
best_model = compare_models(sort = 'MAPE', exclude= ['lightgbm'])

In [ ]:
#Choosing Best Model - K=3-Cross validation, et- Extra Trees Regressor
rf = create_model('et', fold = 3)

In [ ]:
#Plotting the model (Hyper parameter optimization)
# We are selecting best optimized parameters of extra trees regresser on the basis of average MAPE(mean absolute percentage error) 
tuned_dt = tune_model(rf, optimize = 'MAPE')

In [ ]:
plot_model(rf, plot = 'residuals')